# Example 3: Binary Mixture with a Tangent Pinch

#### A mixture of Benzene and Ethylenediamine has an additional constraint on the Reflux Ratio due to a pinch point on the equilibrium curve

In [1]:
## IMPORT NECESSARY LIBRARIES ##
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from thermo_models.RaoultsLawModel import *
from distillation.DistillationModel import DistillationModel
from distillation.DistillationBinary import DistillationModelBinary
from IPython.display import clear_output
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from thermo_models.VLEEmpiricalModel import *
from IPython.display import display
from ipywidgets import interactive, Button, VBox, HBox, Output
from IPython.display import clear_output
import matplotlib.pyplot as plt


In [2]:

#Define empirical relationship
def xtoy(x):
    a, b = 9, -0.6
    y = a*x/(1+(a-1)*x) +b*x*(1 - x)
    return y

# Create a Raoult's law object
vle_model = VLEEmpiricalModelBinary(xtoy)

# Define the sliders:
xD1_slider = widgets.FloatSlider(value=0.95, min=0.5, max=0.999, step=0.0001, description='$x_D$:', readout_format='.2f')
xB1_slider = widgets.FloatSlider(value=0.05, min=0, max=0.5, step=0.0001, description='$x_B$:', readout_format='.2f')
R1_slider =  widgets.FloatLogSlider(value=1.0, base=10, min=-1, max=1, step=0.001,  description='Reflux Ratio:', readout_format='.1f')
R2_slider =  widgets.FloatSlider(value=1.00, min=0.8, max=1.2, step=0.0001, description='Reflux Ratio:', readout_format='.2f')

def plot_distil_oplines_ex2(xD=xD1_slider.value, xB=xB1_slider.value, R=R1_slider.value):
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one.
    distillation_model = DistillationModelBinary(vle_model, xF = np.array([0.5, 0.5]), xD = np.array([xD, 1 - xD]), xB = np.array([xB, 1 - xB]), reflux = R)
    fig, axs = plt.subplots(2, 3, figsize=(15, 5), gridspec_kw={'height_ratios': [40, 1]}, sharex='col')
    distillation_model.plot_distil_strip_binary(axs[0][0], axs[1][0])
    distillation_model.plot_distil_rect_binary(axs[0][1], axs[1][1])
    distillation_model.plot_distil_binary(axs[0][2], axs[1][2])
    plt.subplots_adjust(hspace=0)
    plt.show()  # Ensure the plot is displayed in Jupyter.

def plot_distil_stripping_ex2(xD=xD1_slider.value, xB=xB1_slider.value, R=R1_slider.value):
    plt.close('all')  # Ensure that any existing figures are closed.
    fig, axs = plt.subplots(2, 1, figsize=(7, 7), gridspec_kw={'height_ratios': [40, 1]}, sharex='col')
    distillation_model = DistillationModelBinary(vle_model, xF=np.array([0.5, 0.5]), xD=np.array([xD, 1 - xD]), xB=np.array([xB, 1 - xB]), reflux=R)
    distillation_model.plot_distil_strip_binary(ax=axs[0], ax_fixed=axs[1])
    plt.subplots_adjust(hspace=0)
    plt.show()  # This is essential for the plot to be displayed in the Output widget in Jupyter.
    
def plot_distil_rectifying_ex2(xD=0.95, xB=0.05, R=R2_slider.value):
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one.
    distillation_model = DistillationModelBinary(vle_model, xF=np.array([0.5, 0.5]), xD=np.array([xD, 1 - xD]), xB=np.array([xB, 1 - xB]), reflux=R)
    fig, axs = plt.subplots(2, 2, figsize=(14, 7), gridspec_kw={'height_ratios': [40, 1]}, sharex='col')

    distillation_model.plot_distil_rect_binary(ax=axs[0, 0], ax_fixed=axs[1, 0])
    #distillation_model.plot_distil_rect_binary(ax=axs[0, 1], ax_fixed=axs[1, 1])

    #distillation_model.plot_distil_rect_binary(ax=axs[0, 0].inset_axes([0.6, 0.6, 0.35, 0.35]), ax_fixed=axs[1, 0].inset_axes([0.6, 0.6, 0.35, 0.35]))
    distillation_model.plot_distil_rect_binary(ax=axs[0, 1], ax_fixed=axs[1, 1], zoom_factor = 1)
    
    plt.subplots_adjust(hspace=0, wspace=0.4)  # Adjust spacing between subplots
    plt.show()  # Ensure the plot is displayed in Jupyter.
    
out_strip = Output()
out_rect = Output()
out_oplines = Output()
'''
# Define the sliders:
xD2_slider = widgets.FloatSlider(value=0.95, min=0.5, max=0.999, step=0.0001, description='$x_D$:', readout_format='.2f')
xB2_slider = widgets.FloatSlider(value=0.05, min=0, max= 0.5, step=0.0001, description='$x_B$:', readout_format='.2f')
r_max_slider = widgets.FloatSlider(value=1.0, min=0, max=100, step=1, description='R max:', readout_format='.1f')

def plot_r_vs_fixedpoints_ex2(r_max = r_max_slider.value, xD=xD2_slider.value, xB=xB2_slider.value):
    clear_output(wait=True)
    display("Calculating... Please wait.")
    r_values = np.linspace(0.001, r_max, 100)
    x_fixed_r_arr = []
    x_fixed_s_arr = []

    for r in tqdm(r_values, desc="Progress"):
        distillation_model = DistillationModelBinary(vle_model, xF=np.array([0.5, 0.5]), xD=np.array([xD, 1 - xD]), xB=np.array([xB, 1 - xB]), reflux=r)
        x_fixed_r, _ = distillation_model.find_rect_fixedpoints_binary(100)
        x_fixed_s, _ = distillation_model.find_strip_fixedpoints_binary(100)
        x_fixed_r_arr.append(x_fixed_r)
        x_fixed_s_arr.append(x_fixed_s)

    # flatten the lists and repeat the r_values according to the number of x values
    r_values_r = np.repeat(r_values, [len(x) for x in x_fixed_r_arr])
    r_values_s = np.repeat(r_values, [len(x) for x in x_fixed_s_arr])
    x_fixed_r_arr = np.hstack(x_fixed_r_arr)
    x_fixed_s_arr = np.hstack(x_fixed_s_arr)

    plt.scatter(r_values_r, x_fixed_r_arr,marker = 'x')
    plt.scatter(r_values_s, x_fixed_s_arr, marker ='x')
    plt.xlabel("Reflux Ratio")
    plt.ylabel("x")
    plt.show()
'''

'\n# Define the sliders:\nxD2_slider = widgets.FloatSlider(value=0.95, min=0.5, max=0.999, step=0.0001, description=\'$x_D$:\', readout_format=\'.2f\')\nxB2_slider = widgets.FloatSlider(value=0.05, min=0, max= 0.5, step=0.0001, description=\'$x_B$:\', readout_format=\'.2f\')\nr_max_slider = widgets.FloatSlider(value=1.0, min=0, max=100, step=1, description=\'R max:\', readout_format=\'.1f\')\n\ndef plot_r_vs_fixedpoints_ex2(r_max = r_max_slider.value, xD=xD2_slider.value, xB=xB2_slider.value):\n    clear_output(wait=True)\n    display("Calculating... Please wait.")\n    r_values = np.linspace(0.001, r_max, 100)\n    x_fixed_r_arr = []\n    x_fixed_s_arr = []\n\n    for r in tqdm(r_values, desc="Progress"):\n        distillation_model = DistillationModelBinary(vle_model, xF=np.array([0.5, 0.5]), xD=np.array([xD, 1 - xD]), xB=np.array([xB, 1 - xB]), reflux=r)\n        x_fixed_r, _ = distillation_model.find_rect_fixedpoints_binary(100)\n        x_fixed_s, _ = distillation_model.find_strip

###  Demo 2: Rectifying Pinch Region

##### The inflection point in the equilibrum curve restricts the range of feasible reflux rations when the distillate composition is sufficiently high (set at 0.95 for this Demo)

In [3]:
# Create your interactive widget but don't display it
rect_plot = interactive(plot_distil_rectifying_ex2, xD=0.95, xB=0.05, R=R2_slider, continuous_update=False)

# Define your button click behavior
def on_rect_button_click(b):
    with out_rect:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_distil_rectifying_ex2(xD=0.95, xB=0.05, R=R2_slider.value)

rect_button = Button(description='Generate Plot')
rect_button.on_click(on_rect_button_click)

layout = VBox([R2_slider, rect_button, out_rect])
display(layout)

In [4]:
# Create your interactive widget but don't display it
oplines_plot = interactive(plot_distil_oplines_ex2, xD=xD1_slider, xB=xB1_slider, R=R1_slider, continuous_update=False)

# Define your button click behavior
def on_oplines_button_click(b):
    with out_oplines:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_distil_oplines_ex2(xD=xD1_slider.value, xB=xB1_slider.value, R=R1_slider.value)

oplines_button = Button(description='Generate Oplines Plot')
oplines_button.on_click(on_oplines_button_click)

layout_oplines = VBox([HBox(oplines_plot.children[:-1]), oplines_button, out_oplines])
display(layout_oplines)

In [5]:
# # Create the interactive widget:
# fixed_points_reflux = widgets.interactive(plot_r_vs_fixedpoints_ex2, xD=xD2_slider, xB=xB2_slider, r_max = r_max_slider) 

# # Display the interactive widget:
# display(fixed_points_reflux)

In [6]:
from scipy.misc import derivative

